In [2]:
import io
from surprise import Dataset
from surprise import KNNWithMeans
from surprise import get_dataset_dir
from collections import defaultdict

k=4
similarFilms = 5
userID = input("Enter user ID: ")

data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
simOptions = {'name':'cosine','min_support':similarFilms}

alg=KNNWithMeans(k=k, min_k=k, sim_options = simOptions)
alg.fit(trainset)

testSet = trainset.build_anti_testset()
testSet = filter(lambda x: x[0] == userID, testSet)
predictions = alg.test(testSet)
topN = defaultdict(list)

for uid, iid, _, est, _ in predictions:
    topN[uid].append((iid, round(est, 3)))

for uid, userRnk in topN.items():
    userRnk.sort(key=lambda x: x[1], reverse=True)
    topN[uid] = userRnk[:similarFilms]

file = get_dataset_dir() + '/ml-100k/ml-100k/u.item'
data = {}
rowFormat = "{:^4} {:<95} {:^1}"

with open(file, 'r', encoding = "ISO-8859-1") as f:
    for line in f:
        line = line.split('|')
        data[line[0]] = (line[1], line[2])

print(f'User {userID}:')
films = []
for movieID, rating in topN[userID]:
    print(rowFormat.format(movieID,str(data[movieID]),rating))
    films.append(data[movieID][0])

Enter user ID: 9
Computing the cosine similarity matrix...
Done computing similarity matrix.
User 9:
222  ('Star Trek: First Contact (1996)', '22-Nov-1996')                                              5
 32  ('Crumb (1994)', '01-Jan-1994')                                                                 5
515  ('Boot, Das (1981)', '04-Apr-1997')                                                             5
288  ('Scream (1996)', '20-Dec-1996')                                                                5
403  ('Batman (1989)', '01-Jan-1989')                                                                5


In [12]:
from SPARQLWrapper import SPARQLWrapper, JSON
from IPython.display import display, HTML
import pandas as pd
import requests
import re

sparql = SPARQLWrapper("https://query.wikidata.org/sparql", agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36')
API_ENDPOINT = "https://www.wikidata.org/w/api.php"

In [16]:
def Sparql(sparql, film):
    print(film)
    queryString = """
SELECT ?countryLabel ?filmLabel
WHERE {
wd:"""+film+""" wdt:P495 ?country . #ищем страну фильма
?film wdt:P31 wd:Q11424 .
?film wdt:P495 ?country .
SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". }
}
limit 100
    """
    sparql.setQuery(queryString)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    results_df = pd.io.json.json_normalize(results['results']['bindings'])
    return results_df

In [17]:
print("Список фильмов, снятых в той же стране, что и рекомендованный фильм")
# выводим результаты запроса для каждого фильма в списке рекомендаций
for f in films:
    film = re.sub('\([0-9]{4}\)', "", f) # убираем год
    params = {'action' : 'wbsearchentities','format' : 'json','language' : 'en','search': film}
    res = requests.get(API_ENDPOINT, params = params)
    print(film,":")
    #проверяем, нашелся ли фильм
    if res.json()['search']:
        #если фильм найден, то запускаем выполнение запроса
        results_df = Sparql(sparql, res.json()['search'][0]['id'])
        #проверяем, есть ли результаты выполнения запроса 
        if len(results_df.columns) > 0:
            results_df = results_df.reindex(columns=['countryLabel.value', 'filmLabel.value'], fill_value='no data')
            display(HTML(results_df[['countryLabel.value', 'filmLabel.value']].to_html()))
        else:
            print("Нет результатов\n")
    else:
        print("Фильм не найден\n")

Список фильмов, снятых в той же стране, что и рекомендованный фильм
Star Trek: First Contact  :
Q221236


,countryLabel.value,filmLabel.value
0,United States of America,The Last Airbender
1,United States of America,Eraserhead
2,United States of America,E.T. the Extra-Terrestrial
3,United States of America,Swing Vote
4,United States of America,Hackers
5,United States of America,The Day After Tomorrow
6,United States of America,Oxygen
7,United States of America,The Sisterhood of the Traveling Pants 2
8,United States of America,Thursday
9,United States of America,Star Wars Episode IV: A New Hope


Crumb  :
Q2619747


,countryLabel.value,filmLabel.value
0,United States of America,Dances with Wolves
1,United States of America,Signs
2,United States of America,Fear and Loathing in Las Vegas
3,United States of America,Africa Screams
4,United States of America,Abbott and Costello in the Foreign Legion
5,United States of America,Godzilla
6,United States of America,Alligator II - The Mutation
7,United States of America,Visions of Light
8,United States of America,Jaded
9,United States of America,We Live in Public


Boot, Das  :
Фильм не найден

Scream  :
Q471379
Нет результатов

Batman  :
Q80370
Нет результатов

